In [11]:
import pandas as pd 
from dataclasses import dataclass
from typing import List
from datetime import datetime

In [12]:
combined_df= pd.read_excel(r"..\Data\Report_data_final.xlsx",index_col=None)
combined_df = combined_df.rename(columns={'Unnamed: 0': 'Session_Run_ID'})

In [13]:
@dataclass
class Rat(): 
    id:int
    Name:str
    birth_date:datetime 
    age:float
    threshold:int
    hole_dist:List[dict]
    # accuracy:float 

In [14]:
def rat_profiling(row): 
 
    age=list(set(row['Age']))
    gender= list(set(row['Gender']))

    row['Age'] = age
    row['Gender']= gender
    
    

In [34]:




def get_values(lst):
    # Ensure it's a list
    if isinstance(lst, list):
        # Check if all elements are NaN-like
        if all(pd.isna(x) for x in lst):
            return ['Unknown']
    return list(set(lst))


def _rate(lst): 
    return sum(lst)/len(lst)

def _get_median(lst):
    n= len(lst)
    lst.sort()
    if n % 2 == 0:
        res = (lst[n//2 - 1] + lst[n//2]) / 2
    else:
        res = lst[n//2]

    return res


def rat_profiling(row): 
 
    age=get_values(row['Age'])[-1]
    gender= get_values(row['Gender'])[0]
    weight= get_values(row['Weight'])[0]
    Threshold_Time = _get_median(row['Threshold_Time'])
    sniff_time= _rate(row['Sniff_Time'])


    TP = _rate(row['TP'])
    FP = _rate(row['FP'])
    TN = _rate(row['TN'])
    FN = _rate(row['FN'])


    Accuracy = (sum(row['TP']) + sum(row['TN']))/(sum(row['TP']) + sum(row['TN']) + sum(row['FP']) + sum(row['FN']))
    Recall = (sum(row['TP'])/((sum(row['TP']))+ sum(row['FN'])))



    row['Age'] = age
    row['Gender']= gender
    row['Weight'] = weight
    row['Threshold_Time']= Threshold_Time
    row['Sniff_Time'] = sniff_time
    row['TP'] = TP
    row['FP'] = FP
    row['TN'] = TN
    row['FN']= FN
    row['Accuracy'] = Accuracy
    row['Recall'] = Recall

    

    return row
    
    
by_rat=combined_df.groupby('Rat_Name')[['Gender','Age','Weight','Sniff_Time','TP','FP','TN','FN','Threshold_Time']].agg(list)
by_rat = by_rat.reset_index()
rat_profiled= by_rat.apply(lambda x: rat_profiling(x),axis=1)

In [35]:
rat_profiled.to_excel(r"..\Data\Rat_profiles.xlsx")